## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [3]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

week_ending = "Dec. 1st"
week_starting = "Nov. 24th"

print(week_starting, week_ending)

Nov. 24th Dec. 1st


In [4]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-26-2024,Condo/Co-op,15125 Michelangelo Blvd #208,Delray Beach,FL,33446.0,340500.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/15125-M...,Beaches MLS,RX-11040402,N,Y,26.450333,-80.158449


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [9]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [10]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
14,PAST SALE,2024-11-27,Condo/Co-op,341 Lake Frances Dr,West Palm Beach,FL,33411.0,130000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/341-...,Beaches MLS,RX-11029290,N,Y,26.703005,-80.163756
35,PAST SALE,2024-11-27,Condo/Co-op,1815 S Olive Ave #2,West Palm Beach,FL,33401.0,534000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1815...,Beaches MLS,RX-11013185,N,Y,26.697471,-80.052656
25,PAST SALE,2024-11-27,Condo/Co-op,15365 Lakes Of Delray Blvd #102,Delray Beach,FL,33484.0,315000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/15365-L...,Beaches MLS,RX-11017471,N,Y,26.450335,-80.133666
4,PAST SALE,2024-11-27,Condo/Co-op,250 S Ocean Blvd #251,Delray Beach,FL,33483.0,1100000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/250-S-O...,Beaches MLS,F10471180,N,Y,26.456355,-80.060002
30,PAST SALE,2024-11-27,Condo/Co-op,1551 N Flagler Dr Unit Uph-7,West Palm Beach,FL,33401.0,570000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1551...,Beaches MLS,RX-11017939,N,Y,26.727245,-80.051353
6,PAST SALE,2024-11-27,Condo/Co-op,1745 Palm Cove Blvd Unit 3-204,Delray Beach,FL,33445.0,215000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1745-Pa...,Beaches MLS,RX-11028247,N,Y,26.438248,-80.103785
50,PAST SALE,2024-11-27,Condo/Co-op,5839 Parkwalk Dr,Boynton Beach,FL,33472.0,224900.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/5839-P...,Beaches MLS,RX-10962136,N,Y,26.556510,-80.136082
49,PAST SALE,2024-11-27,Condo/Co-op,215 Saxony E,Delray Beach,FL,33446.0,85000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/215-Sax...,Beaches MLS,RX-10964756,N,Y,26.440263,-80.150386
33,PAST SALE,2024-11-27,Condo/Co-op,3490 Briar Bay Blvd #203,West Palm Beach,FL,33411.0,230000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3490...,Beaches MLS,RX-11014450,N,Y,26.728279,-80.137427
11,PAST SALE,2024-11-27,Condo/Co-op,6169 Balboa Cir #103,Boca Raton,FL,33433.0,235000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/6169-Balb...,Beaches MLS,RX-11030384,N,Y,26.345058,-80.143613


In [11]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [12]:
df_top_ten = df.head(10)

## Collect Agent Information

In [13]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [14]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [15]:
response_df = pd.DataFrame(response_list)

In [16]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [17]:
df_top_ten = merged_df

## Current Week's Values

In [18]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

53


In [19]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$334,751


In [20]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$276


In [21]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$17,741,790


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [22]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
82
Input Previous Week Condo Average Sales Price:
386830
Input Previous Week Condo Average PSF:
288
Input Previous Week Condo Sales total (ex: 198_000_000)
31_700_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-25,Condo/Co-op,2660 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,2750000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1980.0,NaN,1287.0,2798.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11019070,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Christopher Leavitt,Douglas Elliman,William McManus,The Corcoran Group,Jonathan Buch,Redfin Corporation,None,None,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-25,Condo/Co-op,2660 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,2750000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1980.0,NaN,1287.0,2798.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11019070,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Christopher Leavitt,Douglas Elliman,William McManus,The Corcoran Group,Jonathan Buch,Redfin Corporation,None,None,1,orange
1,PAST SALE,2024-11-26,Condo/Co-op,5380 N Ocean Dr Unit 20J,Singer Island,FL,33404.0,1475000.0,3.0,3.5,EASTPOINTE CONDO,2715.0,NaN,1981.0,NaN,543.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,MARMLS,A11675526,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Paige Rose,Weigert Realty Services Corp.,None,None,None,None,None,None,2,blue
2,PAST SALE,2024-11-27,Condo/Co-op,250 S Ocean Blvd #251,Delray Beach,FL,33483.0,1100000.0,2.0,2.0,Windemere House,1400.0,NaN,1974.0,NaN,786.0,899.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/250-S-O...,Beaches MLS,F10471180,N,Y,26.456355,-80.060002,https://www.redfin.com/FL/Delray-Beach/250-S-O...,David Lawrence,Home Hunters Real Estate,None,None,David Lawrence,Home Hunters Real Estate,None,None,3,blue
3,PAST SALE,2024-11-25,Condo/Co-op,1701 S Flagler Dr #803,West Palm Beach,FL,33401.0,975000.0,2.0,2.0,Rapallo Condo,1350.0,NaN,1973.0,NaN,722.0,1202.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,Beaches MLS,RX-10938342,N,Y,26.698765,-80.050982,https://www.redfin.com/FL/West-Palm-Beach/1701...,Rosalind Clarke,"Premier Estate Properties, Inc",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,4,blue
4,PAST SALE,2024-11-27,Condo/Co-op,1551 N Flagler Dr Unit Uph-7,West Palm Beach,FL,33401.0,570000.0,2.0,2.0,The Slade,1175.0,NaN,2005.0,NaN,485.0,1411.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1551...,Beaches MLS,RX-11017939,N,Y,26.727245,-80.051353,https://www.redfin.com/FL/West-Palm-Beach/1551...,John Rupp,Premier Brokers International,None,None,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue
5,PAST SALE,2024-11-26,Condo/Co-op,3212 S Ocean Blvd Unit 302-A,Highland Beach,FL,33487.0,563500.0,2.0,2.0,Seagate OF Highland Condo,1201.0,NaN,1970.0,NaN,469.0,973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3212-...,Beaches MLS,RX-11003336,N,Y,26.417108,-80.065899,https://www.redfin.com/FL/Highland-Beach/3212-...,Derek West,Coldwell Banker,None,None,Jeff Daly,Realty Home Advisors Inc,None,None,6,blue
6,PAST SALE,2024-11-27,Condo/Co-op,1815 S Olive Ave #2,West Palm Beach,FL,33401.0,534000.0,2.0,2.5,Villas EL Centro Condo,1175.0,NaN,1985.0,NaN,454.0,750.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1815...,Beaches MLS,RX-11013185,N,Y,26.697471,-80.052656,https://www.redfin.com/FL/West-Palm-Beach/1815...,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,7,blue
7,PAST SALE,2024-11-26,Condo/Co-op,1701 S Flagler Dr #806,West Palm Beach,FL,33401.0,445000.0,1.0,1.5,Rapallo Condo,1005.0,NaN,1973.0,NaN,443.0,857.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,Beaches MLS,RX-11028675,N,Y,26.698765,-80.050982,https://www.redfin.com/FL/West-Palm-Beach/1701...,Hadar Goldberg PA,Keller Williams Realty - Welli,Robert Yacovone,Keller Williams Realty - Welli,Linda Essig,Illustrated Properties,None,None,8,blue
8,PAST SALE,2024-11-26,Condo/Co-op,405 Sea Oats Dr Unit B,Juno Beach,FL,33408.0,435000.0,2.0,2.0,Sea Oats of Juno 

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-25,Condo/Co-op,2660 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,2750000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1980.0,NaN,1287.0,2798.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11019070,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Christopher Leavitt,Douglas Elliman,William McManus,The Corcoran Group,Jonathan Buch,Redfin Corporation,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_12022024


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'Beach Point Condo at 2660 S Ocean Blvd Unit 501n in Palm Beach'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-25,Condo/Co-op,2660 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,2750000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1980.0,NaN,1287.0,2798.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11019070,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Christopher Leavitt,Douglas Elliman,William McManus,The Corcoran Group,Jonathan Buch,Redfin Corporation,None,None,1,orange,Beach Point Condo at 2660 S Ocean Blvd Unit 50...
1,PAST SALE,2024-11-26,Condo/Co-op,5380 N Ocean Dr Unit 20J,Singer Island,FL,33404.0,1475000.0,3.0,3.5,EASTPOINTE CONDO,2715.0,NaN,1981.0,NaN,543.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,MARMLS,A11675526,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Paige Rose,Weigert Realty Services Corp.,None,None,None,None,None,None,2,blue,EASTPOINTE CONDO at 5380 N Ocean Dr Unit 20J i...
2,PAST SALE,2024-11-27,Condo/Co-op,250 S Ocean Blvd #251,Delray Beach,FL,33483.0,1100000.0,2.0,2.0,Windemere House,1400.0,NaN,1974.0,NaN,786.0,899.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/250-S-O...,Beaches MLS,F10471180,N,Y,26.456355,-80.060002,https://www.redfin.com/FL/Delray-Beach/250-S-O...,David Lawrence,Home Hunters Real Estate,None,None,David Lawrence,Home Hunters Real Estate,None,None,3,blue,Windemere House at 250 S Ocean Blvd #251 in De...
3,PAST SALE,2024-11-25,Condo/Co-op,1701 S Flagler Dr #803,West Palm Beach,FL,33401.0,975000.0,2.0,2.0,Rapallo Condo,1350.0,NaN,1973.0,NaN,722.0,1202.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,Beaches MLS,RX-10938342,N,Y,26.698765,-80.050982,https://www.redfin.com/FL/West-Palm-Beach/1701...,Rosalind Clarke,"Premier Estate Properties, Inc",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,4,blue,Rapallo Condo at 1701 S Flagler Dr #803 in Wes...
4,PAST SALE,2024-11-27,Condo/Co-op,1551 N Flagler Dr Unit Uph-7,West Palm Beach,FL,33401.0,570000.0,2.0,2.0,The Slade,1175.0,NaN,2005.0,NaN,485.0,1411.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1551...,Beaches MLS,RX-11017939,N,Y,26.727245,-80.051353,https://www.redfin.com/FL/West-Palm-Beach/1551...,John Rupp,Premier Brokers International,None,None,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue,The Slade at 1551 N Flagler Dr Unit Uph-7 in W...
5,PAST SALE,2024-11-26,Condo/Co-op,3212 S Ocean Blvd Unit 302-A,Highland Beach,FL,33487.0,563500.0,2.0,2.0,Seagate OF Highland Condo,1201.0,NaN,1970.0,NaN,469.0,973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3212-...,Beaches MLS,RX-11003336,N,Y,26.417108,-80.065899,https://www.redfin.com/FL/Highland-Beach/3212-...,Derek West,Coldwell Banker,None,None,Jeff Daly,Realty Home Advisors Inc,None,None,6,blue,Seagate OF Highland Condo at 3212 S Ocean Blvd...
6,PAST SALE,2024-11-27,Condo/Co-op,1815 S Olive Ave #2,West Palm Beach,FL,33401.0,534000.0,2.0,2.5,Villas EL Centro Condo,1175.0,NaN,1985.0,NaN,454.0,750.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1815...,Beaches MLS,RX-11013185,N,Y,26.697471,-80.052656,https://www.redfin.com/FL/West-Palm-Beach/1815...,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,7,blue,Villas EL Centro Condo at 1815 S Olive Ave #2 ...
7,PAST SALE,2024-11-26,Condo/Co-op,1701 S Flagler Dr #806,West Palm Beach,FL,33401.0,445000.0,1.0,1.5,Rapallo Condo,1005.0,NaN,1973.0,NaN,443.0,857.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,B

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Beach Point Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $380,000M to $2,750,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach, Juno Beach, Singer Island, Delray Beach, West Palm Beach, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 53 condo sales totaling $17,741,790 million from Nov. 24th to Dec. 1st. The previous week, brokers closed 82 condo sales totaling $31,700,000.

Last week’s units sold for an average of $334,751, lower than the $3

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-25,Condo/Co-op,2660 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,2750000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1980.0,NaN,1287.0,2798.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11019070,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Christopher Leavitt,Douglas Elliman,William McManus,The Corcoran Group,Jonathan Buch,Redfin Corporation,None,None,1,orange,Beach Point Condo at 2660 S Ocean Blvd Unit 50...
1,PAST SALE,2024-11-26,Condo/Co-op,5380 N Ocean Dr Unit 20J,Singer Island,FL,33404.0,1475000.0,3.0,3.5,EASTPOINTE CONDO,2715.0,NaN,1981.0,NaN,543.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,MARMLS,A11675526,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Paige Rose,Weigert Realty Services Corp.,None,None,None,None,None,None,2,blue,EASTPOINTE CONDO at 5380 N Ocean Dr Unit 20J i...
2,PAST SALE,2024-11-27,Condo/Co-op,250 S Ocean Blvd #251,Delray Beach,FL,33483.0,1100000.0,2.0,2.0,Windemere House,1400.0,NaN,1974.0,NaN,786.0,899.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/250-S-O...,Beaches MLS,F10471180,N,Y,26.456355,-80.060002,https://www.redfin.com/FL/Delray-Beach/250-S-O...,David Lawrence,Home Hunters Real Estate,None,None,David Lawrence,Home Hunters Real Estate,None,None,3,blue,Windemere House at 250 S Ocean Blvd #251 in De...
3,PAST SALE,2024-11-25,Condo/Co-op,1701 S Flagler Dr #803,West Palm Beach,FL,33401.0,975000.0,2.0,2.0,Rapallo Condo,1350.0,NaN,1973.0,NaN,722.0,1202.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,Beaches MLS,RX-10938342,N,Y,26.698765,-80.050982,https://www.redfin.com/FL/West-Palm-Beach/1701...,Rosalind Clarke,"Premier Estate Properties, Inc",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,4,blue,Rapallo Condo at 1701 S Flagler Dr #803 in Wes...
4,PAST SALE,2024-11-27,Condo/Co-op,1551 N Flagler Dr Unit Uph-7,West Palm Beach,FL,33401.0,570000.0,2.0,2.0,The Slade,1175.0,NaN,2005.0,NaN,485.0,1411.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1551...,Beaches MLS,RX-11017939,N,Y,26.727245,-80.051353,https://www.redfin.com/FL/West-Palm-Beach/1551...,John Rupp,Premier Brokers International,None,None,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue,The Slade at 1551 N Flagler Dr Unit Uph-7 in W...
5,PAST SALE,2024-11-26,Condo/Co-op,3212 S Ocean Blvd Unit 302-A,Highland Beach,FL,33487.0,563500.0,2.0,2.0,Seagate OF Highland Condo,1201.0,NaN,1970.0,NaN,469.0,973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3212-...,Beaches MLS,RX-11003336,N,Y,26.417108,-80.065899,https://www.redfin.com/FL/Highland-Beach/3212-...,Derek West,Coldwell Banker,None,None,Jeff Daly,Realty Home Advisors Inc,None,None,6,blue,Seagate OF Highland Condo at 3212 S Ocean Blvd...
6,PAST SALE,2024-11-27,Condo/Co-op,1815 S Olive Ave #2,West Palm Beach,FL,33401.0,534000.0,2.0,2.5,Villas EL Centro Condo,1175.0,NaN,1985.0,NaN,454.0,750.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1815...,Beaches MLS,RX-11013185,N,Y,26.697471,-80.052656,https://www.redfin.com/FL/West-Palm-Beach/1815...,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,7,blue,Villas EL Centro Condo at 1815 S Olive Ave #2 ...
7,PAST SALE,2024-11-26,Condo/Co-op,1701 S Flagler Dr #806,West Palm Beach,FL,33401.0,445000.0,1.0,1.5,Rapallo Condo,1005.0,NaN,1973.0,NaN,443.0,857.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,B

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Beach Point Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $380,000M to $2,750,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach, Juno Beach, Singer Island, Delray Beach, West Palm Beach, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 53 condo sales totaling $17,741,790 million from Nov. 24th to Dec. 1st. The previous week, brokers closed 82 condo sales totaling $31,700,000.

Last week’s units sold for an average of $334,751, lower than the $3

In [56]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Palm-Beach/2660-S-Ocean-Blvd-33480/unit-501N/home/42641716


In [57]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Riviera-Beach/5380-N-Ocean-Dr-33404/unit-20J/home/42460213


In [58]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Delray-Beach/825-Egret-Cir-33444/unit-210/home/42372681


In [59]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Juno-Beach/405-Sea-Oats-Dr-33408/unit-B/home/42390774


In [60]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/2660-S-Ocean-Blvd-33480/unit-501N/home/42641716


In [61]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Delray-Beach/825-Egret-Cir-33444/unit-210/home/42372681


In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-25,Condo/Co-op,2660 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,2750000.0,2.0,2.5,Beach Point Condo,2137.0,NaN,1980.0,NaN,1287.0,2798.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-11019070,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Christopher Leavitt,Douglas Elliman,William McManus,The Corcoran Group,Jonathan Buch,Redfin Corporation,None,None,1,orange,Beach Point Condo at 2660 S Ocean Blvd Unit 50...
1,PAST SALE,2024-11-26,Condo/Co-op,5380 N Ocean Dr Unit 20J,Singer Island,FL,33404.0,1475000.0,3.0,3.5,EASTPOINTE CONDO,2715.0,NaN,1981.0,NaN,543.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,MARMLS,A11675526,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Paige Rose,Weigert Realty Services Corp.,None,None,None,None,None,None,2,blue,EASTPOINTE CONDO at 5380 N Ocean Dr Unit 20J i...
2,PAST SALE,2024-11-27,Condo/Co-op,250 S Ocean Blvd #251,Delray Beach,FL,33483.0,1100000.0,2.0,2.0,Windemere House,1400.0,NaN,1974.0,NaN,786.0,899.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/250-S-O...,Beaches MLS,F10471180,N,Y,26.456355,-80.060002,https://www.redfin.com/FL/Delray-Beach/250-S-O...,David Lawrence,Home Hunters Real Estate,None,None,David Lawrence,Home Hunters Real Estate,None,None,3,blue,Windemere House at 250 S Ocean Blvd #251 in De...
3,PAST SALE,2024-11-25,Condo/Co-op,1701 S Flagler Dr #803,West Palm Beach,FL,33401.0,975000.0,2.0,2.0,Rapallo Condo,1350.0,NaN,1973.0,NaN,722.0,1202.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,Beaches MLS,RX-10938342,N,Y,26.698765,-80.050982,https://www.redfin.com/FL/West-Palm-Beach/1701...,Rosalind Clarke,"Premier Estate Properties, Inc",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,4,blue,Rapallo Condo at 1701 S Flagler Dr #803 in Wes...
4,PAST SALE,2024-11-27,Condo/Co-op,1551 N Flagler Dr Unit Uph-7,West Palm Beach,FL,33401.0,570000.0,2.0,2.0,The Slade,1175.0,NaN,2005.0,NaN,485.0,1411.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1551...,Beaches MLS,RX-11017939,N,Y,26.727245,-80.051353,https://www.redfin.com/FL/West-Palm-Beach/1551...,John Rupp,Premier Brokers International,None,None,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue,The Slade at 1551 N Flagler Dr Unit Uph-7 in W...
5,PAST SALE,2024-11-26,Condo/Co-op,3212 S Ocean Blvd Unit 302-A,Highland Beach,FL,33487.0,563500.0,2.0,2.0,Seagate OF Highland Condo,1201.0,NaN,1970.0,NaN,469.0,973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3212-...,Beaches MLS,RX-11003336,N,Y,26.417108,-80.065899,https://www.redfin.com/FL/Highland-Beach/3212-...,Derek West,Coldwell Banker,None,None,Jeff Daly,Realty Home Advisors Inc,None,None,6,blue,Seagate OF Highland Condo at 3212 S Ocean Blvd...
6,PAST SALE,2024-11-27,Condo/Co-op,1815 S Olive Ave #2,West Palm Beach,FL,33401.0,534000.0,2.0,2.5,Villas EL Centro Condo,1175.0,NaN,1985.0,NaN,454.0,750.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1815...,Beaches MLS,RX-11013185,N,Y,26.697471,-80.052656,https://www.redfin.com/FL/West-Palm-Beach/1815...,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,Lisa Mastronardi,"Sotheby's Intl. Realty, Inc.",None,None,7,blue,Villas EL Centro Condo at 1815 S Olive Ave #2 ...
7,PAST SALE,2024-11-26,Condo/Co-op,1701 S Flagler Dr #806,West Palm Beach,FL,33401.0,445000.0,1.0,1.5,Rapallo Condo,1005.0,NaN,1973.0,NaN,443.0,857.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1701...,B

## Time on Market Calculator

In [64]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 10, 13) ## List (Earlier) date
date2 = datetime(2024, 11, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

44


## Clean CSV for Datawrapper Chart

In [65]:
chart_df = df_top_ten

In [66]:
chart_df = chart_df.fillna(" ")

In [67]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [68]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [72]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [74]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-25,2660 S Ocean Blvd Unit 501n in Palm Beach,"$2,750,000",2,2.5,"2,137",1980,"$1,287",Beaches MLS,RX-11019070,Christopher Leavitt Douglas Elliman William Mc...,Jonathan Buch Redfin Corporation
1,2024-11-26,5380 N Ocean Dr Unit 20J in Singer Island,"$1,475,000",3,3.5,"2,715",1981,$543,MARMLS,A11675526,Paige Rose Weigert Realty Services Corp.,
2,2024-11-27,250 S Ocean Blvd #251 in Delray Beach,"$1,100,000",2,2.0,"1,400",1974,$786,Beaches MLS,F10471180,David Lawrence Home Hunters Real Estate,David Lawrence Home Hunters Real Estate
3,2024-11-25,1701 S Flagler Dr #803 in West Palm Beach,"$975,000",2,2.0,"1,350",1973,$722,Beaches MLS,RX-10938342,"Rosalind Clarke Premier Estate Properties, Inc...",Suzanne Trapani Frisbie The Corcoran Group
4,2024-11-27,1551 N Flagler Dr Unit Uph-7 in West Palm Beach,"$570,000",2,2.0,"1,175",2005,$485,Beaches MLS,RX-11017939,John Rupp Premier Brokers International,"Andrew Leibowitz Leibowitz Realty Group, LLC./..."
5,2024-11-26,3212 S Ocean Blvd Unit 302-A in Highland Beach,"$563,500",2,2.0,"1,201",1970,$469,Beaches MLS,RX-11003336,Derek West Coldwell Banker,Jeff Daly Realty Home Advisors Inc
6,2024-11-27,1815 S Olive Ave #2 in West Palm Beach,"$534,000",2,2.5,"1,175",1985,$454,Beaches MLS,RX-11013185,"Lisa Mastronardi Sotheby's Intl. Realty, Inc.","Lisa Mastronardi Sotheby's Intl. Realty, Inc."
7,2024-11-26,1701 S Flagler Dr #806 in West Palm Beach,"$445,000",1,1.5,"1,005",1973,$443,Beaches MLS,RX-11028675,Hadar Goldberg PA Keller Williams Realty - Wel...,Linda Essig Illustrated Properties
8,2024-11-26,405 Sea Oats Dr Unit B in Juno Beach,"$435,000",2,2.0,"1,390",1987,$313,Beaches MLS,RX-10955920,Susan Dephouse PA Platinum Properties/The Keye...,Jeanette Romano CGI Realty
9,2024-11-27,825 Egret Cir #210 in Delray Beach,"$380,000",2,2.0,"1,272",1982,$299,Beaches MLS,RX-11018922,Delia Velez Cardinal Group Enterprise INC,Edithann Bradway The Keyes Company
